In [ ]:
# 1. Force Start Docker Service
print("🔧 [1/4] Starting System Services...")
!apt-get update -qq > /dev/null
!apt-get install -y docker.io adb -qq > /dev/null

# This part fixes the 'Cannot connect' error
!service docker start

# 2. Launch Android Engine
print("📱 [2/4] Booting Android...")
!docker run -d --privileged -p 5555:5555 redroid/redroid:11.0.0-latest

# 3. Launch Web Interface
print("🌐 [3/4] Opening Web Portal...")
!npx -y ws-scrcpy-docker --port 7860 > /dev/null 2>&1 &

# 4. Generate Cloudflare Link
print("🔗 [4/4] Creating Link... (Waiting 15 seconds)")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null

import os
import subprocess
import threading
import time

def run_tunnel():
    os.system("cloudflared tunnel --url http://localhost:7860")

threading.Thread(target=run_tunnel, daemon=True).start()

time.sleep(15)
print("\n🚀 READY! SEARCH BELOW FOR THE '.trycloudflare.com' LINK")